# Nanosafety Data Interface Survey

Nanosafety data interface – online user interface enabling user friendly access to the aggregated search index of (sub)set of eNanoMapper database instances. Usually the user interface is project specific and protected but can be also publicly available. Multiple project specific interfaces at https://search.data.enanomapper.net/.


In [ ]:
import yaml
from pynanomapper import aa
from pynanomapper import units
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import requests
from importlib import reload 
from pynanomapper import client_solr
from pynanomapper import client_ambit
from pynanomapper import annotation
import pandas as pd
import numpy as np
import json
import pandas as pd
import importlib
import warnings
import qgrid 

import matplotlib.pyplot as plt
import io
from measurement.utils import guess
from measurement.measures import Distance, Temperature, Volume, Area, Frequency, Mass, Time
#init_notebook_mode(connected=True)
warnings.simplefilter("ignore")

import logging,os,time
from logging.config import fileConfig
fileConfig('logging_endpoints_config.ini')
logger = logging.getLogger()
logger.info('Started at %s \t%s',os.name, time.time())

In [ ]:
print('Select enanoMapper aggregated search service:')
style = {'description_width': 'initial'}
config,config_servers, config_security, auth_object, msg = aa.parseOpenAPI3()    
service_widget = widgets.Dropdown(
    options=config_servers['url'],
    description='Service:',
    disabled=False,
    style=style
)
if config_security is None:
    service = interactive(aa.search_service_open,url=service_widget)
else:
    print(msg)
    apikey_widget=widgets.Text(
            placeholder='',
            description=config_security,
            disabled=False,
            style=style
    )    
    service = interactive(aa.search_service_protected,url=service_widget,apikey=apikey_widget)    

display(service)

In [ ]:
service_uri=service_widget.value
if auth_object!=None:
    auth_object.setKey(apikey_widget.value)
print("Sending queries to {}".format(service_uri))

In [ ]:
def log_query(q):
    #logger.debug(q)
    pass
    
def beautify(r):
    pass
facets = client_solr.Facets()    

# Database records

In [ ]:
dfm = facets.summary(service_uri,auth_object, query="*:*".format(project),fields=["owner_name_hs"],fq='type_s:substance',log_query=log_query,log_result=beautify)    
projects = dfm["owner_name_hs"].unique()
dfm

In [ ]:
#projects = ["NanoReg2","OMICS_DATA","NANoREG","NanoTest","ENPRA","MARINA","NanoGenotox","SANOWORK","caNanoLab","eNanoMapper","GRACIOUS"]
for project in projects:
    dfm = facets.summary(service_uri,auth_object, query="owner_name_hs:{}".format(project),fields=["owner_name_hs","substanceType_hs","s_uuid_hs","publicname_hs","name_hs"],fq='type_s:substance',log_query=log_query,log_result=beautify)    

    if dfm.shape[0]>0:
        dfm.to_csv("materials_{}.csv".format(project))  
        dfs = facets.summary(service_uri,auth_object, query="owner_name_s:{}".format(project),fields=["owner_name_s","document_uuid_s"],fq='type_s:study',log_query=log_query,log_result=beautify)    

        dfs.to_csv("study_{}.csv".format(project))
        print("Project `{}` Materials {} Studies {} Data {}".format(project,dfm.shape[0],dfs.shape[0],dfs["Number of data points"].sum()))


# Endpoints

In [ ]:

facets = client_solr.Facets()
df = facets.summary(service_uri,auth_object, query="*:*",fields=["topcategory_s","endpointcategory_s"],log_query=log_query,log_result=beautify)    

df.rename(columns={'owner_name_s':'project','endpointcategory_name' : 'endpoint', 'topcategory_s' : 'category'},inplace=True)
df.to_csv("endpoints_all.csv")  
#qgrid.show_grid(df[["category","endpoint","Number of data points"]])
df[["category","endpoint","Number of data points"]]

In [ ]:
facets = client_solr.Facets()
df = facets.summary(service_uri,auth_object, query="*:*",fields=["topcategory_s","endpointcategory_s","owner_name_s"],log_query=log_query,log_result=beautify)    

df.rename(columns={'owner_name_s':'project','endpointcategory_name' : 'endpoint', 'topcategory_s' : 'category'},inplace=True)

In [ ]:
(df[["project","category","endpoint","Number of data points"]])

# Material types

In [ ]:

for project in projects:
    dfm = facets.summary(service_uri,auth_object, query="owner_name_hs:{}".format(project),fields=["owner_name_hs","substanceType_hs"],fq='type_s:substance',log_query=log_query,log_result=beautify)    
    if dfm.shape[0]>0:
        dfm.to_csv("materialtypes_{}.csv".format(project))  
        print("Project `{}` Material types {}".format(project,dfm.shape[0]))
        #display(dfm)

In [ ]:
# All material types
dfm = facets.summary(service_uri,auth_object, query="*:*".format(project),fields=["substanceType_hs"],fq='type_s:substance',log_query=log_query,log_result=beautify)    
dfm.to_csv("materialtypes_all.csv".format(project))  
dfm    